In [23]:
!nvidia-smi

Sun Feb 27 17:35:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W /  70W |  14556MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
from google.colab import auth
auth.authenticate_user()

In [25]:
!gcloud config set project alecsharpie
!gcloud config list

Updated property [core/project].
[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
account = alecsharpie@gmail.com
project = alecsharpie

Your active configuration is: [default]


In [26]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   117k      0 --:--:-- --:--:-- --:--:--  117k
OK
69 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.40.0).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 69 not upgraded.


In [27]:
!mkdir gcs_bucket
!gcsfuse --implicit-dirs image-datasets-alecsharpie gcs_bucket

mkdir: cannot create directory ‘gcs_bucket’: File exists
2022/02/27 17:35:13.557418 Start gcsfuse/0.40.0 (Go version go1.17.6) for app "" using mount point: /content/gcs_bucket
2022/02/27 17:35:13.571485 Opening GCS connection...
2022/02/27 17:35:13.702817 Mounting file system "image-datasets-alecsharpie"...
2022/02/27 17:35:13.703560 File system has been successfully mounted.


In [28]:
!ls gcs_bucket

cassava_farmer	packages


In [29]:
!ls gcs_bucket

cassava_farmer	packages


In [30]:
!pip install git+https://github.com/alecsharpie/cassava_farmer.git --no-cache


  Cloning https://github.com/alecsharpie/cassava_farmer.git to /tmp/pip-req-build-wcikl4r7
  Running command git clone -q https://github.com/alecsharpie/cassava_farmer.git /tmp/pip-req-build-wcikl4r7


In [31]:
from cassava_farmer.trainer import Trainer

In [32]:
!mkdir history
!mkdir models

mkdir: cannot create directory ‘history’: File exists
mkdir: cannot create directory ‘models’: File exists


In [33]:
!pwd

/content


In [34]:
# trainer = Trainer('colab')
# trainer.train(data_path = 'gcs_bucket/cassava_farmer/train_images')

In [35]:
!zip -r /content/models/aug_eff_model_test_colab2.zip /content/models/aug_eff_model_test

from google.colab import files
files.download("/content/models/aug_eff_model_test_colab2.zip")

	zip warning: name not matched: /content/models/aug_eff_model_test

zip error: Nothing to do! (try: zip -r /content/models/aug_eff_model_test_colab2.zip . -i /content/models/aug_eff_model_test)


FileNotFoundError: ignored

In [ ]:
import matplotlib.pyplot as plt
import json

history = json.load(open('/content/history/history_2022-02-27_16-43-50'))

history

In [ ]:
plt.plot()

In [39]:
# home made model

from tensorflow.keras import layers
from tensorflow.keras import Sequential

def build_model():
  model = Sequential([
                      layers.Conv2D(16, (5, 5), activation = 'relu', input_shape = (512, 512, 3)),
                      layers.MaxPooling2D(4),
                      layers.Conv2D(32, (5, 5), activation = 'relu'),
                      layers.MaxPooling2D(4),
                      layers.Conv2D(64, (5, 5), activation = 'relu'),
                      layers.MaxPooling2D(4),
                      layers.Flatten(),
                      layers.Dense(10, activation = 'relu'),
                      layers.Dropout(0.4),
                      layers.Dense(5, activation = 'softmax')
  ])

  model.compile(optimizer = 'adam',
                loss = 'sparse_categorical_crossentropy',
                metrics = 'accuracy')
  return model


In [40]:
from tensorflow.keras.utils import image_dataset_from_directory

def get_image_generator_local(
    batch_size,
    train_path='raw_data/cassava-leaf-disease-classification/train_images_mid'
):


    train_ds = image_dataset_from_directory(
        train_path,
        batch_size=batch_size,
        subset='training',
        validation_split=.20,
        seed=42,
        image_size=(512, 512),
    )

    class_names = train_ds.class_names

    train_size = train_ds.cardinality().numpy()
    train_ds = train_ds.unbatch().batch(batch_size)
    train_ds = train_ds.repeat()

    val_ds = image_dataset_from_directory(train_path,
                                          batch_size=32,
                                          subset='validation',
                                          validation_split=.20,
                                          seed=42,
                                          image_size=(512, 512))

    val_size = val_ds.cardinality().numpy()
    val_ds = val_ds.unbatch().batch(batch_size)
    val_ds = val_ds.repeat()
    return train_ds, train_size, val_ds, val_size

batch_size = 1

train_ds, train_size, val_ds, val_size = get_image_generator_local(batch_size, 'gcs_bucket/cassava_farmer/train_images')

steps_per_epoch = train_size // batch_size
validation_steps = val_size // batch_size


Found 21407 files belonging to 5 classes.
Using 17126 files for training.
Found 21407 files belonging to 5 classes.
Using 4281 files for validation.


In [ ]:
label_map = {
    '0': 'cassava_bacterial_blight', #1087
    '1': 'cassava_brown_streak_disease', #2189
    '2': 'cassava_green_mottle', #2386
    '3': 'cassava_mosaic_disease', #13158
    '4': 'healthy' #2577
}

./cassava_mosaic_disease:	   
./healthy:	    
./cassava_brown_streak_disease:	    
./cassava_green_mottle:	    
./cassava_bacterial_blight:	    

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)


In [ ]:
class_weight = {0: weight_for_0, 
                1: weight_for_1,
                2:,
                3:,
                4:}

In [ ]:
test_model = build_model()

history = test_model.fit(
                train_ds,
                epochs=3,
                #batch_size=batch_size,
                steps_per_epoch=steps_per_epoch,
                validation_data=val_ds,
                validation_steps=validation_steps,
                validation_batch_size=batch_size).history
                #callbacks=[es]

Epoch 1/3
10406/17126 [=================>............] - ETA: 9:25 - loss: 1.2284 - accuracy: 0.6200